## Libraries

In [6]:
# Libraries
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.linalg import null_space, det
from tentacles import *
import itertools
import os
from tqdm import tqdm # type: ignore
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import copy
from pymoo.core.problem import ElementwiseProblem # type: ignore
from pymoo.algorithms.moo.nsga2 import NSGA2 # type: ignore
from pymoo.optimize import minimize # type: ignore
from pymoo.visualization.scatter import Scatter # type: ignore
from pymoo.termination import get_termination
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from pymoo.indicators.hv import HV

In [7]:
# Hyperparameters
DISCRETIZATION = 8
NEW_PERTURB = 50
LIMIT = 0.05

## First dataset creation (for coarse optimization)

In [ ]:
L = 0.6
c = 0.5
f = 1
A = np.deg2rad(60)

if True:
    thrust, velocity = NVE(L, c, f, A, plot=True)

KeyboardInterrupt: 

In [ ]:
def simulate_and_extend_dataset(points, dataset_path=None):
    """
    Simulates a list of parameter points and optionally appends the results
    to an existing dataset.

    Args:
        points (np.ndarray): array of shape (N, 4) with parameters [L, c, f, A].
        dataset_path (str or None): if provided, loads and extends the dataset at this path,
                                    otherwise creates a new one.

    Returns:
        inputs_tensor (torch.Tensor): tensor of shape (N_total, 4).
        outputs_tensor (torch.Tensor): tensor of shape (N_total, 2).
    """
    inputs = []
    outputs = []

    # Run simulations
    for L, c, f, A in tqdm(points, desc="Simulating NVE points"):
        try:
            thrust, velocity = NVE(L, c, f, A, plot=False)
            inputs.append([L, c, f, A])
            outputs.append([thrust, velocity])
        except Exception:
            continue  # Skip failed simulation

    # Convert to tensors
    inputs_tensor = torch.tensor(inputs, dtype=torch.float32)
    outputs_tensor = torch.tensor(outputs, dtype=torch.float32)

    # Optionally extend existing dataset
    if dataset_path and os.path.exists(dataset_path):
        existing = torch.load(dataset_path, weights_only=True)
        inputs_tensor = torch.cat([existing["inputs"], inputs_tensor], dim=0)
        outputs_tensor = torch.cat([existing["outputs"], outputs_tensor], dim=0)

    # Save updated dataset
    if dataset_path:
        torch.save({"inputs": inputs_tensor, "outputs": outputs_tensor}, dataset_path)
        print(f"\n Updated dataset saved to {dataset_path}")

In [ ]:
# Discretization for initial grid sampling
discretization = DISCRETIZATION

# Define parameter ranges
L_range = np.linspace(0.1, 0.6, discretization)
c_range = np.linspace(0.0, 1.0, discretization)
f_range = np.linspace(0.01, 3.0, discretization)
A_range_deg = np.linspace(5, 135, discretization)
A_range_rad = np.deg2rad(A_range_deg)

# Generate all parameter combinations
param_grid = np.array(list(itertools.product(L_range, c_range, f_range, A_range_rad)))

# Run simulations and save dataset
if True:
    simulate_and_extend_dataset(
        points=param_grid,
        dataset_path="nve_dataset.pt"
    )

In [ ]:
class NVEModel_norm(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(4, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 2)
        )

    def forward(self, x):
        return self.net(x)

class NVEModel(nn.Module):
    def __init__(self, model, y_min, y_max):
        super().__init__()
        self.model = model
        self.y_min = nn.Parameter(y_min, requires_grad=False)
        self.y_max = nn.Parameter(y_max, requires_grad=False)

    def forward(self, x):
        y_norm = self.model(x)
        return y_norm * (self.y_max - self.y_min) + self.y_min

In [ ]:
def train_nve_model_from_dataset(dataset_path="nve_dataset.pt"):
    """
    Trains a neural network on the normalized NVE dataset.
    Saves the final model if requested. Returns it only if not saving.

    Args:
        dataset_path (str): path to the .pt file containing {"inputs": X, "outputs": y}.
        save_model (bool): if True, saves the trained model to disk and returns nothing.

    Returns:
        model_with_denorm (nn.Module)
    """
    # Load dataset
    data = torch.load(dataset_path, weights_only=True)
    X = data["inputs"]
    y_raw = data["outputs"]

    # Normalize outputs
    y_min = y_raw.min(dim=0).values
    y_max = y_raw.max(dim=0).values
    y_norm = (y_raw - y_min) / (y_max - y_min)

    # Split into training and validation sets
    num_samples = X.shape[0]
    split_idx = int(0.9 * num_samples)
    X_train, X_val = X[:split_idx], X[split_idx:]
    y_train, y_val = y_norm[:split_idx], y_norm[split_idx:]

    # Create DataLoaders
    dataset_train = TensorDataset(X_train, y_train)
    dataset_val = TensorDataset(X_val, y_val)
    loader_train = DataLoader(dataset_train, batch_size=64, shuffle=True)
    loader_val = DataLoader(dataset_val, batch_size=64, shuffle=False)

    model = NVEModel_norm()
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    # Training loop with early stopping
    epochs = 300
    patience = 20
    best_val_loss = float('inf')
    best_model = None
    wait = 0
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        model.train()
        loss_epoch = 0
        for xb, yb in loader_train:
            pred = model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            loss_epoch += loss.item() * xb.size(0)
        train_losses.append(loss_epoch / len(loader_train.dataset))

        model.eval()
        with torch.no_grad():
            val_loss = 0
            for xb, yb in loader_val:
                pred = model(xb)
                val_loss += criterion(pred, yb).item() * xb.size(0)
        val_loss /= len(loader_val.dataset)
        val_losses.append(val_loss)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = copy.deepcopy(model.state_dict())
            wait = 0
        else:
            wait += 1
            if wait >= patience:
                print(f"Early stopping at epoch {epoch}")
                break

        if epoch % 10 == 0:
            print(f"Epoch {epoch:3d} | Train Loss: {train_losses[-1]:.4f} | Val Loss: {val_losses[-1]:.4f}")

    # Load best model weights
    model.load_state_dict(best_model)

    model_with_denorm = NVEModel(model, y_min, y_max)

    # Plot losses
    plt.figure()
    plt.plot(train_losses, label='Train')
    plt.plot(val_losses, label='Validation')
    plt.xlabel('Epoch')
    plt.ylabel('MSE Loss (normalized)')
    plt.legend()
    plt.grid(True)
    plt.title("Training and Validation Loss")
    plt.show()

    # Save or return
    torch.save(model_with_denorm, "nve_model.pt")
    print("Entire model saved to nve_model.pt")

    return model_with_denorm

In [ ]:
# Define the multi-objective problem
class NVEOptimizationProblem(ElementwiseProblem):
    def __init__(self, model):
        super().__init__(n_var=4, n_obj=2, n_constr=0,
                         xl=np.array([0.1, 0.0, 0.01, np.deg2rad(5)]),
                         xu=np.array([0.6, 1.0, 3.0, np.deg2rad(135)]))
        self.model = model

    def _evaluate(self, x, out, *args, **kwargs):
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            pred = self.model(x_tensor).squeeze(0).numpy()

        # Objectives: we want to maximize both → negate them for minimization
        out["F"] = - pred  # [thrust, velocity]

## Active learning for model refinement

In [ ]:
def sample_around_pareto(pareto_points, N_new, sigma, bounds, max_attempts=50):
    """
    Sample points around Pareto front with Gaussian noise, ensuring all samples stay within bounds.
    If sampling fails after `max_attempts`, the original point is used.
    """
    lower, upper = np.array(bounds[0]), np.array(bounds[1])
    M = len(pareto_points)
    n_per_point = max(1, N_new // M)
    samples = []

    for x in pareto_points:
        count = 0
        while count < n_per_point:
            for attempt in range(max_attempts):
                noise = np.random.normal(0, sigma, size=(1, 4))
                perturbed = x + noise
                if np.all(perturbed >= lower) and np.all(perturbed <= upper):
                    samples.append(perturbed[0])
                    break
            else:
                # After max_attempts, use the original point
                samples.append(x)
            count = count + 1

    return np.array(samples)[:N_new]

In [ ]:
def iterative_pareto_refinement(max_iter=200, N_new=NEW_PERTURB, sigma=0.10, dataset_path="nve_dataset.pt", pop_size=100, delta_threshold=LIMIT):
    bounds = (
        np.array([0.1, 0.0, 0.01, np.deg2rad(5)]),  # lower
        np.array([0.6, 1.0, 3.0, np.deg2rad(135)])  # upper
    )


    hv = HV(ref_point=np.array([0.0, 0.0]))  # reference
    prev_hv = None
    delta_hv_history = []

    final_pareto_X = None
    final_pareto_F = None

    for i in range(max_iter):
        print(f"\n=== Iteration {i+1} ===")

        # Train model on current dataset
        model = train_nve_model_from_dataset(dataset_path)

        # Optimize current model with NSGA-II
        problem = NVEOptimizationProblem(model)
        algorithm = NSGA2(pop_size=pop_size)
        res = minimize(problem,
                       algorithm,
                       termination=get_termination("moo"),
                       seed=1,
                       save_history=False,
                       verbose=True)

        # Extract Pareto front
        pareto_X = res.X
        pareto_F = -res.F  # un-negate to get actual thrust and velocity

        # Compute hypervolume
        hv_now = hv(res.F)
        print(f"Hypervolume: {hv_now}")
        if prev_hv is not None:
            delta_hv = np.abs(hv_now - prev_hv)
            delta_hv_history.append(delta_hv)
            print(f"Hypervolume delta: {delta_hv:.6f}")

            # Check plateau over last 5 iterations
            if len(delta_hv_history) >= 5:
                recent_max = np.max(delta_hv_history[-5:])
                print(f"Recent max delta HV: {recent_max:.6f}")
                if recent_max < delta_threshold:
                    print("Plateau detected — stopping iterations.")
                    break
        prev_hv = hv_now

        # Plot current Pareto front
        plt.figure(figsize=(6, 5))
        plt.scatter(
        pareto_F[:, 0] * 1000,
        pareto_F[:, 1] * 100,
        s=50,
        facecolors='none',
        edgecolors='#e2001a',
        linewidths=1.5,
        alpha=0.8
        )
        plt.xlabel("Thrust [mN]", fontsize=12)
        plt.ylabel("Velocity [cm/s]", fontsize=12)
        plt.title(f"Pareto Front - Iteration {i+1}", fontsize=13)
        plt.grid(True, which='both', linestyle='--', linewidth=0.5, alpha=0.7)
        plt.tight_layout()
        plt.show()

        # Store final Pareto front
        final_pareto_X = pareto_X
        final_pareto_F = pareto_F

        # Sample new points around Pareto front
        new_points = sample_around_pareto(pareto_X, N_new=N_new, sigma=sigma, bounds=bounds)

        # Simulate and update dataset
        simulate_and_extend_dataset(new_points, dataset_path=dataset_path)

    
    # Plot delta HV history
    plt.figure()
    plt.plot(range(1, len(delta_hv_history) + 1), delta_hv_history, marker='o')
    plt.xlabel("Iteration")
    plt.ylabel("Delta Hypervolume")
    plt.title("Hypervolume Delta Across Iterations")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

    # Return final Pareto front
    return final_pareto_X, final_pareto_F

In [ ]:
# Application of Pareto front
X_pareto, F_pareto = iterative_pareto_refinement(
    sigma=np.array([0.2, 0.2, 0.4, np.deg2rad(10)])
)
# Save final Pareto front
np.savez("pareto_results.npz", X=X_pareto, F=F_pareto)
print("Saved final Pareto front to pareto_results.npz")

## Statistical analysis of Pareto front

In [ ]:
# Load the trained neural network model
model = torch.load("nve_model.pt")
model.eval()

# Define the multi-objective optimization problem for NSGA-II
class NVEOptimizationProblem(ElementwiseProblem):
    def __init__(self, model):
        super().__init__(
            n_var=4,  # [L, c, f, A]
            n_obj=2,  # [thrust, velocity]
            n_constr=0,
            xl=np.array([0.1, 0.0, 0.01, np.deg2rad(5)]),     # lower bounds
            xu=np.array([0.6, 1.0, 3.0, np.deg2rad(135)])    # upper bounds
        )
        self.model = model

    def _evaluate(self, x, out, *args, **kwargs):
        # Evaluate the model for input x
        x_tensor = torch.tensor(x, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            y_pred = self.model(x_tensor).squeeze().numpy()
        thrust, velocity = y_pred
        out["F"] = [-thrust, -velocity]  # negate to maximize

# Instantiate the problem and optimizer
problem = NVEOptimizationProblem(model)
algorithm = NSGA2(pop_size=100)
termination = get_termination("n_gen", 100)

# Run NSGA-II optimization
res = minimize(problem,
               algorithm,
               termination=termination,
               seed=1,
               save_history=False,
               verbose=True)

# Extract Pareto front and denegate objectives
pareto_X = res.X
pareto_F = -res.F
thrust = pareto_F[:, 0]
velocity = pareto_F[:, 1]
power = thrust * velocity


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import ScalarFormatter

def analyze_pareto_front(F_pareto, X_pareto, model):
    thrusts = F_pareto[:, 0]
    velocities = F_pareto[:, 1]
    powers = thrusts * velocities

    # Max indices
    idx_thrust = thrusts.argmax()
    idx_velocity = velocities.argmax()
    idx_power = powers.argmax()

    print("=== Best by THRUST ===")
    L, c, f, A = X_pareto[idx_thrust]
    print(f"L={L:.3f} m, c={c:.3f}, f={f:.3f} Hz, A={np.rad2deg(A):.1f}° → "
          f"Thrust={thrusts[idx_thrust]:.4f} N, Velocity={velocities[idx_thrust]:.4f} m/s, "
          f"Power={powers[idx_thrust]:.4f} W")

    print("\n=== Best by VELOCITY ===")
    L, c, f, A = X_pareto[idx_velocity]
    print(f"L={L:.3f} m, c={c:.3f}, f={f:.3f} Hz, A={np.rad2deg(A):.1f}° → "
          f"Thrust={thrusts[idx_velocity]:.4f} N, Velocity={velocities[idx_velocity]:.4f} m/s, "
          f"Power={powers[idx_velocity]:.4f} W")

    print("\n=== Best by POWER (Thrust·Velocity) ===")
    L, c, f, A = X_pareto[idx_power]
    print(f"L={L:.3f} m, c={c:.3f}, f={f:.3f} Hz, A={np.rad2deg(A):.1f}° → "
          f"Thrust={thrusts[idx_power]:.4f} N, Velocity={velocities[idx_power]:.4f} m/s, "
          f"Power={powers[idx_power]:.4f} W")

    # Stats
    units = ["N", "m/s", "W"]
    for name, arr, unit in zip(["Thrust", "Velocity", "Power"], [thrusts, velocities, powers], units):
        print(f"\n{name} stats: min={arr.min():.4f} {unit}, max={arr.max():.4f} {unit}, "
              f"mean={arr.mean():.4f} {unit}, std={arr.std():.4f} {unit}, median={np.median(arr):.4f} {unit}")

    # Pareto scatter
    plt.figure(figsize=(8, 6))
    sc = plt.scatter(thrusts, velocities, c=powers, cmap="viridis", edgecolors='k', s=60)
    plt.xlabel("Thrust [N]")
    plt.ylabel("Velocity [m/s]")
    plt.title("Pareto Front Colored by Power")
    cbar = plt.colorbar(sc)
    cbar.set_label("Power [W]")
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

    # Parametric scatter (experimental data)
    param_names = ["L [m]", "c", "f [Hz]", "A [deg]"]
    param_data = [X_pareto[:, 0], X_pareto[:, 1], X_pareto[:, 2], np.rad2deg(X_pareto[:, 3])]

    for name, values in zip(param_names, param_data):
        fig, axs = plt.subplots(1, 2, figsize=(10, 4), sharex=True)
        axs[0].scatter(values, thrusts, color="#1f77b4", edgecolors='k', s=40)
        axs[0].set_ylabel("Thrust [N]")
        axs[0].set_xlabel(name)
        axs[0].grid(True, linestyle='--', alpha=0.6)

        axs[1].scatter(values, velocities, color="#ff7f0e", edgecolors='k', s=40)
        axs[1].set_ylabel("Velocity [m/s]")
        axs[1].set_xlabel(name)
        axs[1].grid(True, linestyle='--', alpha=0.6)

        if name.startswith("f"):
            for ax in axs:
                ax.xaxis.set_major_formatter(ScalarFormatter(useMathText=False))
                ax.xaxis.get_major_formatter().set_scientific(False)

        plt.tight_layout()
        plt.show()

    # NN parametric sweeps — 12 plot grouped in 3 rows (4 subplots each)
    DISCRETIZATION = 100
    L_range = np.linspace(0.1, 0.6, DISCRETIZATION)
    c_range = np.linspace(0.0, 1.0, DISCRETIZATION)
    f_range = np.linspace(0.01, 3.0, DISCRETIZATION)
    A_range_deg = np.linspace(5, 135, DISCRETIZATION)
    A_range_rad = np.deg2rad(A_range_deg)

    param_ranges = [L_range, c_range, f_range, A_range_rad]
    param_labels_plot = ["L [m]", "c [-]", "f [Hz]", "A [deg]"]
    output_idx = {"THRUST": 0, "VELOCITY": 1, "POWER": -1}
    metric_labels = {"THRUST": "Thrust [N]", "VELOCITY": "Velocity [m/s]", "POWER": "Power [W]"}
    param_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']

    best_configs = {
        "THRUST": X_pareto[idx_thrust].tolist(),
        "VELOCITY": X_pareto[idx_velocity].tolist(),
        "POWER": X_pareto[idx_power].tolist(),
    }

    for key in ["THRUST", "VELOCITY", "POWER"]:
        fixed_values = best_configs[key]
        fig, axs = plt.subplots(1, 4, figsize=(16, 3.5), sharey=False)

        for i in range(4):
            x_vals = []
            for val in param_ranges[i]:
                x = fixed_values.copy()
                x[i] = val
                x_vals.append(x)

            X_tensor = torch.tensor(x_vals, dtype=torch.float32)
            with torch.no_grad():
                y_pred = model(X_tensor).numpy()

            if output_idx[key] == -1:
                y_out = y_pred[:, 0] * y_pred[:, 1]
                y_label = "Power [W]"
            else:
                y_out = y_pred[:, output_idx[key]]
                y_label = metric_labels[key]

            x_plot = np.rad2deg(param_ranges[i]) if i == 3 else param_ranges[i]
            x_fixed = np.rad2deg(fixed_values[i]) if i == 3 else fixed_values[i]
            x_label = param_labels_plot[i]

            axs[i].plot(x_plot, y_out, color=param_colors[i])
            axs[i].axvline(x_fixed, color='r', linestyle='--', linewidth=1)
            axs[i].grid(True, linestyle='--', alpha=0.6)
            axs[i].set_xlabel(x_label)
            if param_labels_plot[i] == "f [Hz]":
                axs[i].xaxis.set_major_locator(MaxNLocator(integer=True))
            if i == 0:
                axs[i].set_ylabel(y_label)

        plt.tight_layout()
        plt.show()


In [ ]:
# Load the trained neural network model
model = torch.load("nve_model.pt")
model.eval()

# Analyze the Pareto front
analyze_pareto_front(pareto_F, pareto_X, model)